In [6]:
#This notebook was run on a Google Colab environment using a NVIDIA A100 GPU
!pip install --upgrade pip
!pip install -q -U bitsandbytes

from transformers import AutoProcessor, BitsAndBytesConfig, LlavaForConditionalGeneration, TrainingArguments, Trainer
from PIL import Image
import torch
from peft import LoraConfig, prepare_model_for_kbit_training, get_peft_model, PeftModel
from torch.utils.data import Dataset
from typing import Any, Dict
import random
import requests
import json
import os
from datasets import Dataset, load_dataset

In [2]:
# Load your apod_data.json
with open('/content/drive/MyDrive/APOD_DATA/apod_data.json', 'r') as f:
    apod_data = json.load(f)

# Format data for Hugging Face Dataset
formatted_data = []
image_dir = '/content/drive/MyDrive/APOD_DATA/IMAGES'
for item in apod_data:
    date = item.get('date')
    summary = item.get('explanation')
    if date and summary:
        image_path = os.path.join(image_dir, f"{date}.jpg")
        if os.path.exists(image_path):
            # The prompt structure LLaVA expects
            prompt = f"USER: <image>\nDescribe this image in detail.\nASSISTANT:"
            formatted_data.append({
                "image": image_path,
                "prompt": prompt,
                "summary": summary
            })

# Create the Hugging Face Dataset
dataset = Dataset.from_list(formatted_data)
print("Dataset prepared:")
print(dataset[0])

Dataset prepared:
{'image': '/content/drive/MyDrive/APOD_DATA/IMAGES/2022-12-20.jpg', 'prompt': 'USER: <image>\nDescribe this image in detail.\nASSISTANT:', 'summary': "Thor not only has his own day (Thursday), but a helmet in the heavens.  Popularly called Thor's Helmet, NGC 2359 is a hat-shaped cosmic cloud with wing-like appendages. Heroically sized even for a Norse god, Thor's Helmet is about 30 light-years across. In fact, the cosmic head-covering is more like an interstellar bubble, blown with a fast wind from the bright, massive star near the bubble's center. Known as a Wolf-Rayet star, the central star is an extremely hot giant thought to be in a brief, pre-supernova stage of evolution. NGC 2359 is located about 15,000 light-years away toward the constellation of the Great Overdog. This remarkably sharp image is a mixed cocktail of data from  narrowband filters, capturing not only natural looking stars but details of the nebula's filamentary structures. The star in the center o

In [3]:
# --- Load the Model and Processor ---
print("Loading the LLaVA model and processor...")
model_id = "llava-hf/llava-1.5-7b-hf"
quantization_config = BitsAndBytesConfig(load_in_4bit=True)

model = LlavaForConditionalGeneration.from_pretrained(
    model_id,
    quantization_config=quantization_config,
    device_map="auto"
)
processor = AutoProcessor.from_pretrained(model_id)
processor.tokenizer.pad_token = processor.tokenizer.eos_token

# --- Configure PEFT for LoRA Fine-Tuning ---
lora_config = LoraConfig(
    r=16, lora_alpha=32, lora_dropout=0.05,
    bias="none", target_modules=["q_proj", "k_proj", "v_proj", "o_proj"]
)
model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

# --- Create a Custom Dataset and Data Collator ---
class LLaVADataset(torch.utils.data.Dataset):
    def __init__(self, hf_dataset):
        self.dataset = hf_dataset

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        item = self.dataset[idx]
        image = Image.open(item["image"])
        full_text = item["prompt"] + " " + item["summary"]
        return {"image": image, "text": full_text}

class LLaVADataCollator:
    def __init__(self, processor):
        self.processor = processor

    def __call__(self, features):
        images = [f["image"] for f in features]
        texts = [f["text"] for f in features]

        # The batch should be returned as CPU tensors.
        # The Trainer will automatically move it to the GPU.
        batch = self.processor(
            text=texts,
            images=images,
            return_tensors="pt",
            padding=True
        )

        batch["labels"] = batch["input_ids"]
        return batch

# --- Set up and Run the Trainer ---
print("Configuring and starting the training...")
training_args = TrainingArguments(
    output_dir="llava_apod_finetuned",
    learning_rate=1.4e-5,
    num_train_epochs=1,
    per_device_train_batch_size=16,
    gradient_accumulation_steps=1,
    logging_steps=1,
    save_total_limit=1,
    bf16=True,
    dataloader_num_workers=4,
    remove_unused_columns=False,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=LLaVADataset(dataset),
    data_collator=LLaVADataCollator(processor),
)

# Start the fine-tuning process
trainer.train()

print("Fine-tuning complete!")

Loading the LLaVA model and processor...


config.json:   0%|          | 0.00/950 [00:00<?, ?B/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 3 files:   0%|          | 0/3 [00:00<?, ?it/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.18G [00:00<?, ?B/s]

...

/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)


<IPython.core.display.Javascript object>

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: thomaspowell052202 (thomaspowell052202-university-of-new-haven) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss
1,11.304300
2,11.285300
3,11.320100
4,10.790200
5,11.240200
6,10.888000
7,10.923300
8,10.745200
9,10.910600
10,10.422100


Fine-tuning complete!


In [4]:
# Save the fine-tuned adapter weights to a directory
SAVED_MODEL_PATH = "/content/llava-finetuned-apod"
print(f"Saving model to {SAVED_MODEL_PATH}...")
trainer.save_model(SAVED_MODEL_PATH)

# You also need to save the processor
processor.save_pretrained(SAVED_MODEL_PATH)

print("Model and processor saved successfully.")

Saving model to /content/llava-finetuned-apod...
Model and processor saved successfully.


In [7]:
# --- Configuration ---
# Path to the base model on Hugging Face
BASE_MODEL_ID = "llava-hf/llava-1.5-7b-hf"
# Path to the folder where you saved your fine-tuned model
FINETUNED_MODEL_PATH = "/content/llava-finetuned-apod"

# --- Load the Base Model and Processor ---
print("Loading base model and processor...")
# Load the original processor
processor = AutoProcessor.from_pretrained(BASE_MODEL_ID)

# Load the base model in 4-bit for efficiency
model = LlavaForConditionalGeneration.from_pretrained(
    BASE_MODEL_ID,
    load_in_4bit=True,
    device_map="auto"
)

# --- Load and Apply Your Fine-Tuned Weights ---
print(f"Applying fine-tuned LoRA weights from {FINETUNED_MODEL_PATH}...")
# Load the LoRA adapter and merge it into the base model
model = PeftModel.from_pretrained(model, FINETUNED_MODEL_PATH)

# --- Run Inference ---
print("\n--- Running Inference with Fine-Tuned Model ---")
# Example image and prompt
image_url = "https://apod.nasa.gov/apod/image/2212/Thor_Rochford_960.jpg"
image = Image.open(requests.get(image_url, stream=True).raw)
prompt = "USER: <image>\nDescribe this image in detail.\nASSISTANT:"

# Process inputs and generate a response
inputs = processor(text=prompt, images=image, return_tensors="pt").to("cuda")
output = model.generate(**inputs, max_new_tokens=200)

# Decode and print the result
response_text = processor.decode(output[0], skip_special_tokens=True)
print("\nModel Response:")
# The response will include your original prompt, so we clean it up
assistant_response = response_text.split("ASSISTANT:")[-1].strip()
print(assistant_response)

Loading base model and processor...


Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Applying fine-tuned LoRA weights from /content/llava-finetuned-apod...

--- Running Inference with Fine-Tuned Model ---

Model Response:
The image features a beautiful nebula with a blue, pink, and red hue. The nebula is surrounded by a dark background, emphasizing its vibrant colors. The image captures the essence of the cosmos, showcasing the intricate patterns and swirls of the nebula.

There are several stars scattered throughout the image, adding to the overall beauty of the scene. The stars are positioned at various distances from the nebula, creating a sense of depth and dimension in the image.
